In [1]:
from google.colab import auth, files, drive
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import os
from google.cloud import bigquery
client = bigquery.Client(project='datafall-fulfillment')
date = datetime.strftime(datetime.now(), "%m%d%y")

today=datetime.today().strftime('%m_%d_%y')

In [3]:
market_query = """
select distinct id as market_id, name as market,timezone from 
`datafall-prod`.postal_main.core_market 
"""
markets = client.query(market_query).to_dataframe()

date = datetime.strftime(datetime.now(), "%m%d%y")
os.makedirs(date + '_downloads')

In [4]:
courier_query = """
SELECT distinct uuid as courier_uuid, total_jobs, last_job_dt FROM `datafall-prod.postal_main.core_courier` prod

LEFT JOIN 
(select courier_uuid
     , max(job_dt_local) as last_job_dt
  from `postmates-reporting`.stats.job j
 where complete is true
   and date(job_dt_local) >='2018-10-01'
   and date(job_dt_local) < date(current_timestamp())
     and j.fsm_state in ('DidCompleteDropoff','DidAdminCancel','DidCustomerCancel','DidDispatchCancel','DidDispatchRequestException')
    and j.fraud_canceled is false
    and j.charge_state != 'auth_failed'
    and j.is_pickup is false
    and j.is_return_job is false
 group by 1
 order by 1) AS last_job
 
ON prod.uuid=last_job.courier_uuid

where date(date_created)>'2018-09-01'
"""
couriers = client.query(courier_query).to_dataframe()

#Quality Checks
market_query = """
select distinct id as market_id, name,timezone from 
`datafall-prod`.postal_main.core_market 
"""
markets = client.query(market_query).to_dataframe()

today=datetime.today().strftime('%m_%d_%y')

In [39]:
def enrollment(name):
    os.chdir('/content/121120_downloads/')

    incentive_name=name.split('_')[3]
    canvas=str(today)+'_canvas_'+incentive_name+'_v1'
    variant=canvas=str(today)+'_variant_'+incentive_name+'_v1'
    path=os.getcwd()

    df = pd.read_csv(name)
   
    df['courier_uuid']=df['user_id']
    df['canvas_id']=canvas
    df['variant_id']=variant
    df['market_id']=df['Market ID']



    df=df[['courier_uuid','market_id','canvas_id','variant_id']]
    df=df.drop_duplicates()

    df.to_csv(name)

In [41]:
def checks(name):
    df = pd.read_csv(name)
    qc=pd.merge(df,markets,  on=['market_id'])
    qc=pd.merge(qc,couriers,  on='courier_uuid',how='left')
    qc_final = qc.groupby(['name']).agg({'courier_uuid':'count', 'total_jobs':'mean'}).sort_values(by='courier_uuid', ascending=False)#.to_frame().reset_index()

    qc_total = qc.groupby(['canvas_id']).courier_uuid.count().sort_values(ascending=False).to_frame().reset_index()

    l28=qc[qc['last_job_dt']>='2020-11-08']
    l28=l28.groupby('market')['courier_uuid'].count().reset_index()
    l28=l28.rename(columns={"market": "name", 'courier_uuid':'L28'})

    more_than_28=qc[qc['last_job_dt']<'2020-11-08']
    more_than_28=more_than_28.groupby('market')['courier_uuid'].count().reset_index()
    more_than_28=more_than_28.rename(columns={"market": "name", 'courier_uuid':'More than 28'})

    qc_final = qc.groupby(['name']).agg({'courier_uuid':'count', 'total_jobs':'mean'}).sort_values(by='courier_uuid', ascending=False).reset_index()
    qc_final=qc_final.merge(l28, how='left', on='name')
    qc_final=qc_final.merge(more_than_28, how='left', on='name')

    print('-----------------------',name,'-----------------------')

    print( qc_final)
    print (qc_total)

    print('\n','\n','----------------------------------------------------------','\n','\n')


In [42]:
os.chdir("/content/121120_downloads/")
df=pd.read_csv("enr_12_10_20_CFA_v1.csv")

df=df.merge(markets, how='left', on='market_id')
df=df[['canvas_id',	'variant_id',	'courier_uuid',	'market_id', 'name']]

df=df.rename(columns={"name": "market"})
df.to_csv("enr_12_10_20_CFA_v1.csv")

df=pd.read_csv("enr_12_10_20_McD_v1.csv")

df=df.merge(markets, how='left', on='market_id')
df=df[['canvas_id',	'variant_id',	'courier_uuid',	'market_id', 'name']]

df=df.rename(columns={"name": "market"})
df.to_csv("enr_12_10_20_McD_v1.csv")

In [44]:
os.chdir('/content/121120_downloads/')
files=os.listdir()
for file in files:
    if file=='.ipynb_checkpoints':
      pass
    elif file=='.config':
      pass
    else:
      print (file)
      #enrollment(file)
      checks(file)


enr_12_10_20_CFA_v1.csv
----------------------- enr_12_10_20_CFA_v1.csv -----------------------
                   name  courier_uuid  total_jobs     L28  More than 28
0               Atlanta          9606  131.506894  3616.0        4652.0
1         Washington DC          6578  205.602488  2803.0        2583.0
2               Chicago          6116  166.953644  2110.0        2830.0
3    Dallas / Ft. Worth          5688  115.298643  2021.0        3139.0
4               Orlando          5215  100.105106  1859.0        2822.0
..                  ...           ...         ...     ...           ...
139         Anniston AL             2    4.000000     NaN           2.0
140       Waynesboro VA             1         NaN     NaN           NaN
141       Alexandria LA             1   13.000000     1.0           NaN
142         Columbus IN             1   37.000000     1.0           NaN
143         Staunton VA             1    2.000000     NaN           1.0

[144 rows x 5 columns]
                